In [1]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm_notebook as tqdm
import pickle

In [10]:
links = []
for page_num in tqdm(range(2008)):
    response = requests.get(f"https://magyarnemzet.hu/page/{i+1}/?after=2019-01-01&author&before=2019-12-31&category=0&s")
    soup = BeautifulSoup(response.content, 'html.parser')
    titles = soup.select("#left-area h2")
    links.extend([title.find("a").get("href") for title in titles])
with open('links.p', 'wb') as f:
    pickle.dump(links, f)

KeyboardInterrupt: 

In [2]:
with open('links.p', 'rb') as f:
    links = pickle.load(f)

In [3]:
magyar_nemzet_dict = {}
for link in tqdm(links):
    try:
        response = requests.get(link)
    except:
        response.status_code = 500
    if response.status_code == 200:
        related_urls = []
        soup = BeautifulSoup(response.content, 'html.parser')
        try:
            datetime = soup.find("div", {"class": "en-article-dates"}).text.replace("\n", "")
        except:
            datetime = None
        try:
            author = soup.find("div", {"class": "articleSourceAndAuthor"}).text.replace("\n", "").replace("\t", "")
            if "MTI" in author:
                related_urls.append("MTI")
        except:
            author = None
        try:
            paragraphs = soup.find("div", {"class": "entry-content clearfix"}).find_all("p")
            for paragraph in paragraphs:
                if "MTI" in paragraph.text or "mti" in paragraph.text or "Mti" in paragraph.text:
                    related_urls.append("MTI")
                if paragraph.find_all("a") is not None:
                    for link in paragraph.find_all("a"):
                        try:
                            if "http" in link.get('href'):
                                related_urls.append(link.get('href'))
                            else:
                                related_urls.append("https:" + link.get('href'))
                        except:
                            print(paragraph.find_all("a"))

            magyar_nemzet_dict[link] = {
            "related_links" : related_urls,
            "publish_date" : datetime}
        except:
            print(link)
    else:
        print(link, response.status_code)

[<a name="_GoBack"></a>]
https://magyarnemzet.hu/mesterseges-szervek-eloallitasa-a-kutatok-celja-7578956/ 500
https://magyarnemzet.hu/jatekszabalyzat/
https://magyarnemzet.hu/kulfold/a-kormanypart-folenyesen-nyert-az-indiai-parlamenti-valasztasokon-6958574/ 500
[<a name="_GoBack"></a>]



In [5]:
import sys
sys.setrecursionlimit(50000)
with open('magyar_nemzet_dict.p', 'wb') as f:
    pickle.dump(magyar_nemzet_dict, f)